In [1]:
import torch
import numpy as np
import cvxpy as cp
import cvxpylayers

import pickle
import os

from src.neural_net import *
from src.models import *

## Data Preparation

##### Load training data

In [2]:
relative_path = os.getcwd()
relative_path = os.path.abspath("..")
dataset_fn = relative_path + '/robot_nav/data' + '/single.p'
prob_features = ['x0', 'xg']

data_file = open(dataset_fn,'rb')
all_data = pickle.load(data_file)[:100000]  # use only part of the dataset for quick testing
data_file.close()
num_train = len(all_data)
print(f"Number of training samples: {num_train}")

X0 = np.vstack([all_data[ii]['x0'].T for ii in range(num_train)])  
XG = np.vstack([all_data[ii]['xg'].T for ii in range(num_train)])  
OBS = np.vstack([all_data[ii]['xg'].T for ii in range(num_train)])  
XX = np.array([all_data[ii]['XX'] for ii in range(num_train)])
UU = np.array([all_data[ii]['UU'] for ii in range(num_train)])
YY = np.concatenate([all_data[ii]['YY'].astype(int) for ii in range(num_train)], axis=1).transpose(1,0,2)
train_data = [{'x0': X0, 'xg': XG}, {'XX': XX, 'UU' : UU}, YY]


Number of training samples: 83904


In [3]:
print(YY.shape)

for y in YY[0]:
    print(y.shape)

(83904, 3, 20)
(20,)
(20,)
(20,)


In [4]:
for u in UU[0]:
    print(u.shape)

(20,)
(20,)


##### Obstacle Info

In [5]:
Obs_info = np.array([[1.0,  0.0, 0.4, 0.5, 0.0],
                     [0.7, -1.1, 0.5, 0.4, 0.0],
                     [0.4, -2.5, 0.4, 0.5, 0.0]])
n_obs = 3 

##### Dataset Construction

In [6]:
n_features = 6

X_train = train_data[0]  # Problem parameters, will be inputs of the NNs
Y_train = train_data[2]  # Discrete solutions, will be outputs of the NNs
P_train = train_data[1]  # Continuous trajectories, will be used as parameters in training
num_train = Y_train.shape[0]
y_shape = Y_train.shape[1:]
n_y = int(np.prod(y_shape))

feature_blocks = []
for feature in prob_features:
    if feature == "obstacles_map":
        continue
    values = X_train.get(feature)
    if values is None:
        print('Feature {} is unknown or missing'.format(feature))
        continue
    values = np.asarray(values)
    if values.shape[0] != num_train:
        raise ValueError(
            f"Feature '{feature}' has {values.shape[0]} samples, expected {num_train}"
        )
    feature_blocks.append(values.reshape(num_train, -1))
if feature_blocks:
    features = np.concatenate(feature_blocks, axis=1)
else:
    features = np.zeros((num_train, 0))
if features.shape[1] != n_features:
    n_features = features.shape[1]
labels = Y_train.reshape(num_train, n_y)
# print(labels.shape)
# print(labels[:20])
labels_int = labels.astype(np.int64, copy=False)
bit_shifts = np.arange(4 - 1, -1, -1, dtype=np.int64)
outputs_bits = (labels_int[..., None] >> bit_shifts) & 1
# print(outputs_bits.shape)
outputs = outputs_bits.reshape(num_train, -1)

# P_blocks = []
# for sample in data_list:
#     XX = sample["XX"]            # shape (4, H+1) -> [x, y, vx, vy]
#     UU = sample["UU"]            # shape (2, H)
#     # match layer outputs: keep full trajectories and controls together
#     block = np.concatenate(
#         [XX.reshape(-1), UU.reshape(-1)],
#         axis=0,
#     )
#     P_blocks.append(block)

# P_arr = np.stack(P_blocks)       # shape (N, 4*(H+1) + 2*H)

X_arr = features
Y_arr = outputs
P_arr = P_train['XX'][:, :, :]
Pu_arr = P_train['UU'][:, :, :]
# P_arr = np.concatenate([P_train['XX'][:, :, 1:], P_train['UU']], axis=1)

X_tensor = torch.from_numpy(X_arr).float()
Y_tensor = torch.from_numpy(Y_arr).float()
P_tensor = torch.from_numpy(P_arr).float()
Pu_tensor = torch.from_numpy(Pu_arr).float()

In [ ]:
batch_size = 1
dataset = torch.utils.data.TensorDataset(X_tensor, Y_tensor, P_tensor, Pu_tensor)

from torch.utils.data import random_split

n_train = int(0.8 * len(dataset))
n_test = len(dataset) - n_train
train_set, test_set = random_split(dataset, [n_train, n_test])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

## QP Layer

In [8]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="cvxpy.reductions.solvers.solving_chain_utils")

In [9]:
from cvxpy_mpc_layer import *

T = 0.25
H = 20
M = 1  # update if you have more robots
bounds = {
    "x_max": 2.00,
    "x_min": -0.5,
    "y_max": 0.5,
    "y_min": -3.0,
    "v_max": 0.50,
    "v_min": -0.50,
    "u_max": 0.50,
    "u_min": -0.50,
}
weights = (1.0, 1.0, 10.0)  # (Wu, Wp, Wpt)
d_min = 0.25

# Obstacles exactly as in the simulator
from Robots import obstacle
obstacles = [
    obstacle(1.0, 0.0, 0.4, 0.5, 0.0),
    obstacle(0.7, -1.1, 0.5, 0.4, 0.0),
    obstacle(0.40, -2.50, 0.4, 0.5, 0.0),
]

M = 1
p = np.zeros((2, M))  # stack of robot positions; replace with actual state
d_prox = 2.0
coupling_pairs = [
    (m, n)
    for m in range(M)
    for n in range(m + 1, M)
    if np.linalg.norm(p[:, m] - p[:, n]) <= d_prox
]

cplayer, meta = build_mpc_cvxpy_layer(
        T=.25,
        H=20,
        M=1,
        bounds=bounds,
        weights=weights,
        d_min=d_min,
        obstacles=obstacles,
        coupling_pairs=coupling_pairs
    )



/home/x_lab/workspace/l2o/SS_Learning_MIQP_1/.venv/lib/python3.12/site-packages/cvxpy/atoms/affine/reshape.py:68: FutureWarning: 
    You didn't specify the order of the reshape expression. The default order
    used in CVXPY is Fortran ('F') order. This default will change to match NumPy's
    default order ('C') in a future version of CVXPY.
    To suppress this warning, please specify the order explicitly.
    
  warnings.warn(reshape_order_warning, FutureWarning)


In [10]:
for x, y, p, pu in train_loader:
    print(x.shape, y.shape, p.shape)
    
    u_opt, p_opt, v_opt, s_opt = cplayer(
        x[:, 0:2], x[:, 2:4], x[:, 4:6], y
    )
    
    # print(u_opt)
    # print(pu)
    
    threshold = 1e-3
    diff = torch.abs(u_opt - pu).clone()
    diff[diff < threshold] = 0
    print(diff.mean(dim=0))
    
    # print('--------------------------\n\n')
    # # print(p_opt)
    # # print(v_opt)
    # # print(v_opt.shape)
    # # print(p)
    
    # # print(torch.abs(torch.cat([p_opt, v_opt], dim=1) - p))
    
    threshold = 1e-3
    diff = torch.abs(torch.cat([p_opt, v_opt], dim=1) - p).clone()
    diff[diff < threshold] = 0
    print(diff.mean(dim=0))
    
    break

torch.Size([128, 6]) torch.Size([128, 240]) torch.Size([128, 4, 21])
Please consider re-formulating your problem so that it is always solvable or increasing the number of solver iterations.


/home/x_lab/workspace/l2o/SS_Learning_MIQP_1/.venv/lib/python3.12/site-packages/diffcp/cone_program.py:212: SyntaxWarning: invalid escape sequence '\i'
  s \in K                            y \in K^*


SolverError: Solver scs returned status unbounded

## Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
n_input = 6
n_output = 4

nn_model = MLPWithSTE(insize=n_input, outsize=n_output,
                bias=True,
                linear_map=torch.nn.Linear,
                nonlin=nn.ReLU,
                hsizes=[128] * 4)

In [ ]:
Model = SSL_MIQP_incorporated(nn_model, cplayer, 6, 4, device=device)

In [ ]:
from gurobipy import gurobi


training_params = {}
training_params['TRAINING_EPOCHS'] = int(1)
training_params['CHECKPOINT_AFTER'] = int(20)
training_params['LEARNING_RATE'] = 1e-3
training_params['WEIGHT_DECAY'] = 1e-5
training_params['PATIENCE'] = 5

Model.train_SL(ground_truth_solver, train_loader, test_loader, training_params)

NameError: name 'ground_truth_solver' is not defined